In [ ]:
print("In the name of Allah, The Most Gracious and The Most Merciful")

# Importing all modules and csv files

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from fbprophet import Prophet        # I am going to use Prophet <=
import seaborn as sns
import os
import warnings
warnings.filterwarnings("ignore")


for dirname, _, filenames in os.walk('/kaggle/input'): # new way of reading comppetition input  
    for filename in filenames:                         # csv files
        file_name = os.path.join(dirname, filename)    # if you liked, then upvote
        print(file_name)           
        if "train.csv" in file_name:                   # to make notebook popular among Kagglers
            df = pd.read_csv(file_name)                # 'df' represents for train dataset
        elif "test.csv" in file_name:
            test = pd.read_csv(file_name)              # 'test' represents for test dataset
        elif "submission.csv" in file_name:
            sample = pd.read_csv(file_name) 
        elif "holidays.csv" in file_name:
            holidays = pd.read_csv(file_name)

# Formatting DataFrames

In [ ]:
holidays.date = pd.to_datetime(holidays.date) 

In [ ]:
df['date'] = pd.to_datetime(df['date'])
test['date'] = pd.to_datetime(test['date'])
def clean_store(item):
    return item[6:7] # 'KaggleMart' => 'M'
def clean_product(product):
    return product[7:8] # 'Kaggle Hat' => 'H'
def clean_country(items):
    return items[0] # 'Finland' => 'F'
df.store = df.store.apply(clean_store)
test.store = test.store.apply(clean_store)
df['product'] = df['product'].apply(clean_product)
test['product'] = test['product'].apply(clean_product)
df['country'] = df['country'].apply(clean_country)
test['country'] = test['country'].apply(clean_country)
df['hybrid'] =  df['country'] + df['store'] +df['product']
test['hybrid'] =  test['country'] + test['store'] + test['product']
best_df = df.drop([ "country", "store", "product",'row_id'], axis=1) #unnecessary columns 
X_valid = test.drop(["country", "store", "product", 'row_id'], axis=1) #unnecessary columns

**Hybrid** column is combination of first letters of country + store + product

In [ ]:
best_df.hybrid # hybrid is easy way of seperating

In [ ]:
holidays['country'] = holidays['country'].apply(clean_country) # editing country values
holidays = holidays.drop(['event', 'type'], axis=1) # holiday name, event type is not important 

In [ ]:
best_df.rename(columns={'date':'ds', 'num_sold':"y", 'hybrid': 'hybrid'}, inplace=True)
X_valid.rename(columns={'date':'ds', 'hybrid': 'hybrid'}, inplace=True)

Due to Facebook(Meta) Prophet:
* **rename** date => ds
* **rename** num_sold => y 

[More info](https://facebook.github.io/prophet/docs/quick_start.html) 

In [ ]:
holidays.rename(columns={"date": "ds", "country": 'holiday'}, inplace=True)

Separating holidays dataframe into country based holidays

In [ ]:
holidays_no = holidays.loc[(holidays.holiday == 'N')]
holidays_se = holidays.loc[(holidays.holiday == 'S')]
holidays_fi = holidays.loc[(holidays.holiday == 'F')]
holidays = [ holidays_fi, holidays_no, holidays_se] # in hybrid column finland is first, norway is second and sweden is third

In [ ]:
def create_features(df):
    """
    Creates time series features from datetime index.
    """
    df['dayofweek'] = df['ds'].dt.dayofweek
    df['quarter'] = df['ds'].dt.quarter
    df['month'] = df['ds'].dt.month
    df['year'] = df['ds'].dt.year
    df['dayofyear'] = df['ds'].dt.dayofyear
    df['dayofmonth'] = df['ds'].dt.day
    df['weekofyear'] = df['ds'].dt.weekofyear
    

**create_features** function creates aditional features of time series for making model more accurate

In [ ]:
create_features(best_df)
create_features(X_valid)

In [ ]:
best_df.head()

In [ ]:
X_valid.head()

# Prophet model preparation

In [ ]:
def sorting_for_prophet(df, list_base):
    for i in df.hybrid.unique():
        sorted_df = df.loc[(df.hybrid == i)]
        sorted_df.drop("hybrid", axis=1, inplace=True)
        list_base.append(sorted_df)
    return list_base


For each **hybrid** column,
I create seperate prophet model

In [ ]:
valids = [] # valids is 18 different dataframe collection for prediction
trains = [] # trains is 18 diferent dataframe collection for fitting
sorting_for_prophet(X_valid, valids )
sorting_for_prophet(best_df, trains )
print('Sorting is done!')

In [ ]:
trains[0] # this is train dataframe for FMM (Finland Mart Mug)

# Prophet model

In [ ]:
def prophet_each(df, future_df, holidays):
    model = Prophet(holidays=holidays)
    model.fit(df)
    preds = model.predict(future_df)
    return preds.iloc[:, [0, -1]] # return only date and yhat value


**prophet_each** function is implemented  for each hybrid in train dataframe 

holidays is dataframe of holidays dates in each country  

In [ ]:
y_preds = []
for i  in range(18):
    y_preds.append(prophet_each(trains[i], valids[i], holidays=holidays[i%3]))
    print(i)

# Submission 

In [ ]:
num_sold = []
for k in range(365):
    for j in y_preds:
        num_sold.append(j.yhat[k])

Sorting all prediction results into single dataframe

In [ ]:
num_sold[:10]

In [ ]:
sample['row_id'] = sample['row_id'].astype('int32')
sample['num_sold'] = num_sold

In [ ]:
sample.info()

In [ ]:
sample.to_csv('submission_facebook_prophet.csv', index=False)

Share your ideas in the **comments** section